In [2]:
import pandas as pd
import athena_utils as au
import locale
from concurrent.futures import ThreadPoolExecutor, as_completed

locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')  # Windows



'Spanish_Spain.1252'

In [3]:
## PARAMETROS -------------------------------

proyecto = 72
convenio = "54-54-00-00"
hoy = pd.Timestamp.now().normalize()
fecha_inicio = '2025-01-01'
fecha_fin = '2025-12-30'

## -----------------------------------------

In [4]:
query_alumnos = fr"""
 
WITH 
-- ============================================== 
-- filtros de fecha 
-- ============================================== 
parametros AS ( 
    SELECT 
        DATE '2021-01-01' AS fecha_inicio, --just in case 
        CURRENT_DATE AS fecha_fin -- ídem 
)
-- ============================================== 
-- Alumnos por salon
-- ============================================== 
    SELECT distinct
        p.id as id,
		g.institution as institucion,
		g.grade as grado,
        c.id as room_id,
		g.student_id
    FROM datalake.room_roomsessions s 
    CROSS JOIN parametros 
    JOIN datalake.room_room_students b ON s.room_id = b.room_id 
    JOIN datalake.student_student f ON b.student_id = f.id 
    LEFT JOIN datalake.room_room c ON s.room_id = c.id 
    LEFT JOIN datalake.learning_group d ON c.group_id = d.id 
    LEFT JOIN datalake.learning_course e ON d.course_id = e.id 
    LEFT JOIN datalake.enrollment_enrolment g ON f.id = g.student_id AND e.id = g.course_id AND d.id = g.group_id 
    LEFT JOIN datalake.projects p ON g.b2b_project_id = p.id 
    LEFT JOIN datalake.attendance_attendance aa ON aa.room_session_id = s.id AND b.student_id = aa.object_id AND aa.content_type_id = 8 
    WHERE s.start_date BETWEEN parametros.fecha_inicio AND parametros.fecha_fin 
        AND COALESCE(g.state, 'active') NOT IN ('inactive', 'abandoned') 
        AND ( 
            (s.state = 'true' AND aa.status IN (1,2,3,4,5,6)) 
            OR (s.state = 'false' AND s.cancellation_reason_id IN (34,35)) 
        )
		and p.id in ({proyecto})
		

        """

In [5]:
query_cancelaciones = fr"""

WITH
cte_Organization AS (
    SELECT
        poa.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT o.name), ', ') AS org_names
    FROM datalake.project_organization_association poa
    JOIN datalake.organizations o ON poa.organization_id = o.id
    GROUP BY poa.project_id
),
cte_ProgramType AS (
    SELECT
        ppta.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT cbpt.name), ', ') AS program_types
    FROM datalake.project_program_type_association ppta
    JOIN datalake.catalog_b2bprogramtype cbpt ON ppta.program_type_id = cbpt.id
    GROUP BY ppta.project_id
),
base AS (
    SELECT
        DISTINCT
        p.id AS projectsID,
        p.name AS "Proyecto",
        p.internal_name,
        (CASE WHEN ee.institution IS NULL THEN ei.name ELSE ee.institution END) AS institucion,
        ei.municipality as Municipio,
        rr.id AS room,
        rs.id AS sesionID,
        rs.session_number AS sesion,
        ee.grade AS grado,
        (CASE WHEN rs.cancellation_reason_id IS NULL THEN 36 ELSE rs.cancellation_reason_id END) AS reasonID,
        (CASE WHEN rc.name IS NULL THEN 'N/A' ELSE rc.name END) AS Motivo,
        rs.start_date AS "Fecha",
        rs.state AS state,
        
        CONCAT(rraux.name, ' ', rraux.last_name) AS profesor_ied_nombre,
        CASE 
            WHEN aa_ied.status IN (1, 5) THEN true
            ELSE false
        END AS profesor_ied_asistio,
        'boattiniad@gmail.com' AS email_responsable

    FROM
        datalake.room_roomsessions rs
        LEFT JOIN datalake.room_room rr ON rs.room_id = rr.id
        JOIN datalake.enrollment_enrolment ee ON rr.id = ee.room_id AND ee.b2b_project_id IS NOT NULL
        LEFT JOIN datalake.projects p ON (ee.b2b_project_id = p.id OR p.id = rr.project_b2b_id)
        LEFT JOIN cte_Organization o ON p.id = o.project_id
        LEFT JOIN cte_ProgramType pt ON p.id = pt.project_id
        LEFT JOIN datalake.account_user au ON rr.teacher_id = au.id
        LEFT JOIN datalake.catalog_reasonsessioncancellation rc ON (
            (CASE WHEN (rs.state = 'false' AND rs.cancellation_reason_id IS NULL) THEN 36 ELSE rs.cancellation_reason_id END) = rc.id
        )
        LEFT JOIN datalake.educational_institution ei ON ei.id = rr.educational_institution_id
        -- Join para asistencia del profesor IED
        LEFT JOIN datalake.attendance_attendance aa_ied 
            ON aa_ied.room_session_id = rs.id 
            AND aa_ied.room_id = rr.id 
            AND aa_ied.content_type_id = 276
        LEFT JOIN datalake.room_roomauxiliarteacher rraux 
            ON rraux.id = aa_ied.object_id

    WHERE p.id = {proyecto}
)
SELECT *
FROM base
WHERE
    NOT (state = 'false' AND projectsID IN (19, 14) AND (("Fecha" BETWEEN DATE '2024-06-17' AND DATE '2024-07-05') OR ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14')))
    AND
    NOT (state = 'false' AND projectsID IN (47, 48, 56) AND ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14'))
    AND
    (reasonID NOT IN (33, 34, 35) OR reasonID IS NULL)
    AND "Fecha" BETWEEN DATE '{fecha_inicio}' AND DATE '{fecha_fin}';

"""



In [6]:
def ejecutar_queries_paralelo(queries_dict, max_workers=5):
    """
    Ejecuta múltiples queries de Athena en paralelo.
    
    Args:
        queries_dict (dict): Diccionario donde la clave es el nombre identificador 
                            y el valor es la query SQL
                            Ejemplo: {'cancelaciones': query1, 'asistencias': query2}
        max_workers (int): Número máximo de threads paralelos (default: 5)
    
    Returns:
        dict: Diccionario con los DataFrames resultantes
              Ejemplo: {'cancelaciones': df1, 'asistencias': df2}
    """
    results = {}
    errors = {}
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Enviar todas las queries en paralelo
        futures = {
            executor.submit(au.run_athena_query_auto, query, name): name 
            for name, query in queries_dict.items()
        }
        
        # Recoger resultados según vayan terminando
        for future in as_completed(futures):
            query_name = futures[future]
            try:
                df = future.result()
                results[query_name] = df
                print(f"✅ {query_name}: {len(df)} filas obtenidas")
            except Exception as e:
                errors[query_name] = str(e)
                print(f"❌ {query_name}: Error - {e}")
    
    # Reportar resumen
    print(f"\n📊 Resumen: {len(results)}/{len(queries_dict)} queries completadas exitosamente")
    
    if errors:
        print(f"⚠️  Queries con error: {list(errors.keys())}")
    
    return results

In [7]:
# Preparar queries
queries = {
    'cancelaciones': query_cancelaciones,
    'estudiantes': query_alumnos
}

# Ejecutar en paralelo
print("🚀 Iniciando ejecución de queries en paralelo...\n")
dataframes = ejecutar_queries_paralelo(queries)

df_cancelaciones = dataframes.get('cancelaciones')
df_estudiantes = dataframes.get('estudiantes')

🚀 Iniciando ejecución de queries en paralelo...

✅ estudiantes: 7987 filas obtenidas
✅ cancelaciones: 8078 filas obtenidas

📊 Resumen: 2/2 queries completadas exitosamente


In [8]:
# convertir fecha a datetime y extraer mes-año
df_cancelaciones['fecha'] = pd.to_datetime(df_cancelaciones['fecha'])
df_cancelaciones['mes_año'] = df_cancelaciones['fecha'].dt.strftime('%B de %Y')

In [9]:
# Quedarme con el mes_año con 80% de frecuencia, si no, concatenar los 2 más frecuentes ordenados por fecha
mes_ano_counts = df_cancelaciones['mes_año'].value_counts()
total_count = mes_ano_counts.sum()
max_count = mes_ano_counts.max()

if max_count / total_count >= 0.8:
    mes_ano = mes_ano_counts.index[0]
else:
    top_two = mes_ano_counts.head(2)
    # ordenar top_two por fecha
    top_two = top_two.sort_index(key=lambda x: pd.to_datetime(x, format='%B de %Y'))
    mes_ano = f"{top_two.index[0]} y {top_two.index[1]}"

In [10]:
## Metricas por institucion ## 
dia_de_hoy = hoy.strftime('%d de %B de %Y')

# Obtener lista de instituciones únicas
instituciones = df_cancelaciones['institucion'].dropna().unique()

# Diccionario para almacenar métricas por institución
metricas_por_institucion = {}

for institucion in instituciones:
    # Filtrar datos por institución
    df_cancel_inst = df_cancelaciones[df_cancelaciones['institucion'] == institucion]
    df_est_inst = df_estudiantes[df_estudiantes['institucion'] == institucion]
    
    # Calcular métricas para esta institución
    sesiones_programadas = df_cancel_inst['sesionid'].nunique()
    sesiones_ejecutadas = df_cancel_inst[df_cancel_inst['state'] == 'true']['sesionid'].nunique()
    sesiones_dictadas = round(sesiones_ejecutadas / sesiones_programadas, 3) if sesiones_programadas > 0 else 0
    sesiones_asistidas = df_cancel_inst[
        (df_cancel_inst['state'] == 'true') & 
        (df_cancel_inst['profesor_ied_asistio'] == True)
    ]['sesionid'].nunique()
    porcentaje_asistencia = round(sesiones_asistidas / sesiones_ejecutadas, 3) if sesiones_ejecutadas > 0 else 0
    
    estudiantes = df_est_inst['student_id'].nunique()
    cantidad_salones = df_est_inst['room_id'].nunique()
    # lista de grados
    grados = sorted(df_est_inst['grado'].astype(int).unique())
    grados = ', '.join(f"{g}°" for g in grados[:-1]) + f" y {grados[-1]}°"

    # Guardar métricas en el diccionario
    metricas_por_institucion[institucion] = {
        'sesiones_programadas': sesiones_programadas,
        'sesiones_ejecutadas': sesiones_ejecutadas,
        'sesiones_dictadas': sesiones_dictadas,
        'sesiones_asistidas': sesiones_asistidas,
        'porcentaje_asistencia': porcentaje_asistencia,
        'estudiantes': estudiantes,
        'cantidad_salones': cantidad_salones,
        'grados': grados
    }

# Convertir a DataFrame para visualización
df_metricas = pd.DataFrame(metricas_por_institucion).T
df_metricas.index.name = 'institucion'

In [14]:
from jinja2 import Template
from xhtml2pdf import pisa
import os
import re

plantilla_path = 'plantilla.html'

# Cargar plantilla HTML
with open(plantilla_path, 'r', encoding='utf-8') as f:
    plantilla_html = f.read()

# Mostrar variables encontradas en la plantilla (sintaxis Jinja2: {{variable}})
variables_encontradas = re.findall(r'\{\{(\s*\w+\s*)\}\}', plantilla_html)
variables_encontradas = [v.strip() for v in variables_encontradas]
print("Variables encontradas en la plantilla:")
for var in sorted(set(variables_encontradas)):
    print(f"  - {{{{{var}}}}}")

Variables encontradas en la plantilla:
  - {{cantidad_estudiantes}}
  - {{cantidad_salones}}
  - {{cc_docente}}
  - {{convenio}}
  - {{dia_mes_ano}}
  - {{grados}}
  - {{ied}}
  - {{logo_ctc}}
  - {{logo_institucional}}
  - {{mes_ano}}
  - {{nombre_docente}}
  - {{nombre_equipo_ctc}}
  - {{porcentaje_asistencia}}
  - {{sesiones_asistidas}}
  - {{sesiones_dictadas}}
  - {{sesiones_ejecutadas}}
  - {{sesiones_programadas}}


In [15]:
# Crear carpeta de salida
output_folder = 'actas_generadas'
os.makedirs(output_folder, exist_ok=True)

template = Template(plantilla_html)

# Ruta absoluta del logo para que xhtml2pdf pueda resolverlo
logo_ctc = os.path.abspath('logo_ctc.png')

# Generar un PDF por cada institución
for institucion, metricas in metricas_por_institucion.items():
    variables = {
        'ied': institucion,
        'sesiones_programadas': metricas['sesiones_programadas'],
        'sesiones_ejecutadas': metricas['sesiones_ejecutadas'],
        'sesiones_dictadas': f"{metricas['sesiones_dictadas']*100:.1f}%",
        'sesiones_asistidas': metricas['sesiones_asistidas'],
        'porcentaje_asistencia': f"{metricas['porcentaje_asistencia']*100:.1f}%",
        'cantidad_estudiantes': metricas['estudiantes'],
        'cantidad_salones': metricas['cantidad_salones'],
        'dia_mes_ano': dia_de_hoy,
        'mes_ano': mes_ano,
        'convenio': convenio,
        'grados': metricas['grados'],
        'logo_ctc': logo_ctc,
    }

    html_renderizado = template.render(**variables)

    nombre_archivo = f"{institucion.replace('/', '-').replace('\\', '-')}.pdf"
    ruta_salida = os.path.join(output_folder, nombre_archivo)

    with open(ruta_salida, 'wb') as f:
        status = pisa.CreatePDF(html_renderizado, dest=f)

    if status.err:
        print(f"Error: {nombre_archivo}")
    else:
        print(f"Generado: {nombre_archivo}")

print(f"\nSe generaron {len(metricas_por_institucion)} documentos en '{output_folder}'")

Generado: I.E.D. Técnico De Comercio.pdf
Generado: I.E.D. Ntra Señora Del Rosario.pdf
Generado: I.E.D. Nueva Granada.pdf
Generado: I.E.D. Brisas Del Río.pdf
Generado: I.E.D. San Salvador.pdf
Generado: I.E.D. Las Flores.pdf
Generado: I.E.D. Experiencias Pedagógicas.pdf
Generado: I.E.D. Cruzada Social.pdf
Generado: I.E.D. Marie Poussepin.pdf
Generado: I.E.D. María Cano.pdf
Generado: I.E.D. El Campito.pdf
Generado: I.E.D. Fundación Pies Descalzos.pdf
Generado: I.E.D. Comunitaria Metropolitana.pdf
Generado: E.N.S. La Hacienda.pdf
Generado: E.N.S. Del Distrito De Barranquilla.pdf
Generado: I.E.D. San Vicente.pdf
Generado: I.E.D. Villas De San Pablo.pdf
Generado: I.E.D. Alexander Von Humboldt.pdf
Generado: I.E.D. El Cañahuate.pdf
Generado: I.E.D. Hogar Mariano.pdf
Generado: I.E.D. Evardo Turizo.pdf
Generado: I.E.D. Tierra Santa.pdf
Generado: I.E.D. Betania Norte.pdf
Generado: I.E.D. Reuven Feuerstein.pdf
Generado: I.E.D. Manuel Zapata.pdf
Generado: I.E.D. María Inmaculada.pdf
Generado: I.E.D

In [13]:
## ENVIO DE INFORMES POR EMAIL ##
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from dotenv import load_dotenv

# Cargar variables desde .env
load_dotenv()

GMAIL_USER = os.environ['GMAIL_USER']
GMAIL_APP_PASSWORD = os.environ['GMAIL_APP_PASSWORD']

# Construir mapeo institucion -> email_responsable desde df_cancelaciones
email_por_institucion = (
    df_cancelaciones
    .dropna(subset=['institucion', 'email_responsable'])
    .drop_duplicates(subset=['institucion'])
    .set_index('institucion')['email_responsable']
    .to_dict()
)

def enviar_email(destinatario, asunto, cuerpo, archivo_adjunto):
    """Envia un email con un archivo PDF adjunto via Gmail SMTP."""
    msg = MIMEMultipart()
    msg['From'] = GMAIL_USER
    msg['To'] = destinatario
    msg['Subject'] = asunto

    msg.attach(MIMEText(cuerpo, 'plain', 'utf-8'))

    nombre = os.path.basename(archivo_adjunto)
    with open(archivo_adjunto, 'rb') as f:
        adjunto = MIMEApplication(f.read(), _subtype='pdf')
    adjunto.add_header('Content-Disposition', 'attachment', filename=('utf-8', '', nombre))
    msg.attach(adjunto)

    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(GMAIL_USER, GMAIL_APP_PASSWORD)
        server.send_message(msg)

# Enviar un informe a cada institucion
enviados = 0
errores = []

for institucion in metricas_por_institucion:
    email = email_por_institucion.get(institucion)
    if not email:
        errores.append(f"{institucion}: sin email_responsable")
        continue

    nombre_archivo = f"{institucion.replace('/', '-').replace('\\', '-')}.pdf"
    ruta_archivo = os.path.join(output_folder, nombre_archivo)

    if not os.path.exists(ruta_archivo):
        errores.append(f"{institucion}: archivo no encontrado ({nombre_archivo})")
        continue

    asunto = f"Informe {institucion} - {mes_ano}"
    cuerpo = (
        f"Cordial saludo,\n\n"
        f"Adjunto encontrara el informe de asistencia y sesiones de {institucion} "
        f"correspondiente a {mes_ano}.\n\n"
        f"Cualquier inquietud no dude en comunicarse.\n\n"
        f"Atentamente."
    )

    try:
        enviar_email(email, asunto, cuerpo, ruta_archivo)
        enviados += 1
        print(f"Enviado: {institucion} -> {email}")
    except Exception as e:
        errores.append(f"{institucion} ({email}): {e}")
        print(f"Error: {institucion} -> {e}")

print(f"\nResumen: {enviados}/{len(metricas_por_institucion)} emails enviados")
if errores:
    print("Errores:")
    for err in errores:
        print(f"  - {err}")

Enviado: I.E.D. Técnico De Comercio -> boattiniad@gmail.com


KeyboardInterrupt: 